APPENDIX: CHECKING PARAMETERS OF DIFFERENT COMBINATIONS OF CONVOLUTIONS 

In [1]:
!pip install -U tensorboard_plugin_profile
%load_ext tensorboard


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.5 MB/s eta 0:00:00


1.PARENT MODEL- ALEXNET WITH STD CONV


In [2]:
from datetime import datetime
from packaging import version
import tensorflow as tf
import tensorflow_datasets as tfds
import os,time
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import TensorBoard

tfds.disable_progress_bar()

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
print(ds_test)
print(ds_train)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label


def process_images(image, label):
    
    image = tf.image.resize(image, (224,224))
    return image, label

ds_train = ds_train.map(normalize_img)
print(ds_train)
ds_train = ds_train.map(process_images)
print(ds_train)


ds_test = ds_test.map(normalize_img)
ds_test = ds_test.map(process_images)
nclass=10 #10forcifar10

batchsize = [128,
#16,
#32,
#64,
#128
]
for bs in batchsize:
  ds_train = ds_train.batch(bs)
  ds_test = ds_test.batch(bs)
  print(bs)

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
<_PrefetchDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
<_MapDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
128


In [ ]:
#tensorboard = TensorBoard(os.path.join('energylog',f'{batchsize}_{time.time()}'))
from datetime import datetime
from packaging import version

import os
# Create a TensorBoard callback
logs = "alexLCRM-drP1" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '10,20')

1. alexnet
PAREN MODEL

In [3]:
#COMPARING THE ACTUAL ALEX NET 
#alexNet
model1 = keras.models.Sequential([
  keras.layers.Conv2D(filters=64, kernel_size=(11,11), strides=(4,4), input_shape=(224,224,3)),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),


  #keras.layers.DepthwiseConv2D(depth_multiplier=8,kernel_size=5,strides=(1,1),padding='same'),
  keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="same"),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  

  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same"),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same"),
  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="same"),
  #keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  
  #keras.layers.Conv2D(filters=nclass,kernel_size = (1,1), activation='relu', padding="same"), #add number of classes here
  #keras.layers.AvgPool2D(pool_size=(5,5))

  keras.layers.Flatten(),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(100, activation='softmax')
   
  ])
model1.summary()





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 64)        23296     
                                                                 
 batch_normalization (BatchN  (None, 54, 54, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 54, 54, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       409856    
                                                                 
 batch_normalization_1 (Batc  (None, 26, 26, 256)      1

In [ ]:
 # compile statement  when dense layers are removed
 # model.compile(
  #  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #true since no softmax used at output layer
  #  optimizer=keras.optimizers.Adam(learning_rate=3e-4),
   # metrics=["accuracy"], 
#)
#initializer = keras.initializers.he_normal()
model1.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #tensorboard = TensorBoard(os.path.join('energylog',f'{batchsize}_{time.time()}'))
  #@measure_energy(handler=csv_handler, domains=[NvidiaGPUDomain(0)])
  #@measure_energy(handler=csv_handler,domains=[RaplPackageDomain(1), NvidiaGPUDomain(0),RaplDramDomain(0)])
  #def foo():
model1.fit(ds_train,  epochs=50, validation_data=ds_test, callbacks = [tensorboard])
  #for _ in range(1):
   #   foo()
  #..
#model.fit(ds_train,  epochs=10, validation_data=ds_test)
test_loss, test_acc = model1.evaluate(ds_test, verbose=2)

In [ ]:
model1.fit(ds_train,  epochs=26, validation_data=ds_test, callbacks = [tensorboard])
tf.saved_model.save(model1, "saved_model_keras_dir")

TFLITE CONVERSION OF BASE MODEL


In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model1)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

# 2.DWC+PWC : LCRM-ALEXNET (PROPOSED)

In [4]:
# base model obtained after applying LCRM ON AlexNet algorithm
nclasses=10
#nclasses =1000
def my_model2():
    #entry flow
    input = keras.Input(shape=(224,224,3))
    x = layers.Conv2D(64,kernel_size=11,strides=(4,4))(input)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
    
    
    x = layers.DepthwiseConv2D(depth_multiplier=4,kernel_size=5,strides=(1,1),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x1 = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)

      #middle loop
    x = layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same')(x1)
    x = layers.BatchNormalization()(x)
    x2= layers.Activation('relu')(x)

    x = layers.Conv2D(128,kernel_size=1,strides=(1,1),padding='same')(x1)
    x = layers.BatchNormalization()(x)
    x3= layers.Activation('relu')(x)
    x = layers.Concatenate()([x2,x3])

    x = layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    
    #Exit flow
    x = layers.Conv2D(256,kernel_size=1,strides=(1,1),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)

    x=keras.layers.Flatten()(x)
    x=keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)

    x= keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(10, activation='softmax')(x)
    model = keras.Model(input, output)
    return model

model2=my_model2()
model2.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  
  #to check number of parameters:
model2.summary()
 


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 54, 54, 64)   23296       ['input_1[0][0]']                
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 54, 54, 64)  256         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                              

In [ ]:
model2.fit(ds_train,  epochs=26, validation_data=ds_test, callbacks = [tensorboard])
  #for _ in range(1):
   #   foo()
  #..
#model.fit(ds_train,  epochs=10, validation_data=ds_test)
test_loss, test_acc = model2.evaluate(ds_test, verbose=2)

# 3.GROUP CONV + DWS

In [5]:
#nclasses =1000#MODEL 3 without grouped convolution in last (group+ dws)     3
def my_model3():
    #entry flow
    input = keras.Input(shape=(224,224,3))
    x = layers.Conv2D(32,kernel_size=11,strides=(4,4))(input)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
    
    
    x = layers.DepthwiseConv2D(depth_multiplier=8,kernel_size=5,strides=(1,1),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x1 = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
      #middle loop
    x = layers.Conv2D(256,kernel_size=3,strides=(1,1),groups= 256,padding='same')(x1)
    x = layers.BatchNormalization()(x)
    x2= layers.Activation('relu')(x)

    x = layers.Conv2D(128,kernel_size=1,strides=(1,1),groups=128,padding='same')(x1)
    x = layers.BatchNormalization()(x)
    x3= layers.Activation('relu')(x)
    x = layers.Concatenate()([x2,x3])

    x = layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    
    #Exit flow
    x = layers.Conv2D(256,kernel_size=1,strides=(1,1),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)

    x=keras.layers.Flatten()(x)
    x=keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)

    x= keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(10, activation='softmax')(x)
    model = keras.Model(input, output)
    return model



In [6]:
model2=my_model3()
model2.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  
  #to check number of parameters:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_8 (Conv2D)              (None, 54, 54, 32)   11648       ['input_2[0][0]']                
                                                                                                  
 batch_normalization_11 (BatchN  (None, 54, 54, 32)  128         ['conv2d_8[0][0]']               
 ormalization)                                                                                    
                                                                                            

4.GROUP CONV ONLY, GROUP =128

In [8]:
def my_model2(): #nclasses =1000#MODELFULL GROUP)       4
    #entry flow
    input = keras.Input(shape=(224,224,3))
    x = layers.Conv2D(32,kernel_size=11,strides=(4,4))(input)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
    
    
    x = layers.DepthwiseConv2D(depth_multiplier=8,kernel_size=5,strides=(1,1),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x1 = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
      #middle loop
    x = layers.Conv2D(256,kernel_size=3,strides=(1,1),groups= 256,padding='same')(x1)
    x = layers.BatchNormalization()(x)
    x2= layers.Activation('relu')(x)

    x = layers.Conv2D(128,kernel_size=1,strides=(1,1),groups=128,padding='same')(x1)
    x = layers.BatchNormalization()(x)
    x3= layers.Activation('relu')(x)
    x = layers.Concatenate()([x2,x3])

    x = layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    
    #Exit flow
    x = layers.Conv2D(256,kernel_size=1,strides=(1,1),groups=128,padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)

    x=keras.layers.Flatten()(x)
    x=keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)

    x= keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(10, activation='softmax')(x)
    model = keras.Model(input, output)
    return model

model2=my_model2()
model2.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  
  #to check number of parameters:
model2.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_12 (Conv2D)             (None, 54, 54, 32)   11648       ['input_3[0][0]']                
                                                                                                  
 batch_normalization_17 (BatchN  (None, 54, 54, 32)  128         ['conv2d_12[0][0]']              
 ormalization)                                                                                    
                                                                                            

5. POINTWISE CONV ONLY


In [9]:
#pointwise       5
#alexNet
model1 = keras.models.Sequential([
  keras.layers.Conv2D(filters=64, kernel_size=(11,11), strides=(4,4), input_shape=(224,224,3)),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),


  #keras.layers.DepthwiseConv2D(depth_multiplier=8,kernel_size=5,strides=(1,1),padding='same'),
  keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), padding="same"),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  

  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), padding="same"),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), padding="same"),
  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), padding="same"),
  #keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  
  #keras.layers.Conv2D(filters=nclass,kernel_size = (1,1), activation='relu', padding="same"), #add number of classes here
  #keras.layers.AvgPool2D(pool_size=(5,5))

  keras.layers.Flatten(),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(10, activation='softmax')
   
  ])
model1.summary()

 # compile statement  when dense layers are removed
 # model.compile(
  #  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #true since no softmax used at output layer
  #  optimizer=keras.optimizers.Adam(learning_rate=3e-4),
   # metrics=["accuracy"], 
#)
#initializer = keras.initializers.he_normal()






Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 54, 54, 64)        23296     
                                                                 
 batch_normalization_23 (Bat  (None, 54, 54, 64)       256       
 chNormalization)                                                
                                                                 
 activation_23 (Activation)  (None, 54, 54, 64)        0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 26, 26, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 26, 26, 256)       16640     
                                                                 
 batch_normalization_24 (Bat  (None, 26, 26, 256)     

In [ ]:
model1.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #tensorboard = TensorBoard(os.path.join('energylog',f'{batchsize}_{time.time()}'))
  #@measure_energy(handler=csv_handler, domains=[NvidiaGPUDomain(0)])
  #@measure_energy(handler=csv_handler,domains=[RaplPackageDomain(1), NvidiaGPUDomain(0),RaplDramDomain(0)])
  #def foo():
model1.fit(ds_train,  epochs=25, validation_data=ds_test, callbacks = [tensorboard])
  #for _ in range(1):
   #   foo()
  #..
#model.fit(ds_train,  epochs=10, validation_data=ds_test)
test_loss, test_acc = model1.evaluate(ds_test, verbose=2)

6. DILATED CONVOLUTION ONLY

In [10]:
#out = Conv2D(10, (3, 3), dilation_rate=2)(input_tensor)   6
#dilated convolution full

model1 = keras.models.Sequential([
  keras.layers.Conv2D(filters=64, kernel_size=(11,11), strides=(4,4), input_shape=(224,224,3)),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),


  #keras.layers.DepthwiseConv2D(depth_multiplier=8,kernel_size=5,strides=(1,1),padding='same'),
  keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="same",dilation_rate=2),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  

  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same",dilation_rate=2),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same",dilation_rate=2),
  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="same",dilation_rate=2),
  #keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  
  #keras.layers.Conv2D(filters=nclass,kernel_size = (1,1), activation='relu', padding="same"), #add number of classes here
  #keras.layers.AvgPool2D(pool_size=(5,5))

  keras.layers.Flatten(),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(10, activation='softmax')
   
  ])
model1.summary()





Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 54, 54, 64)        23296     
                                                                 
 batch_normalization_28 (Bat  (None, 54, 54, 64)       256       
 chNormalization)                                                
                                                                 
 activation_28 (Activation)  (None, 54, 54, 64)        0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 26, 26, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 26, 26, 256)       409856    
                                                                 
 batch_normalization_29 (Bat  (None, 26, 26, 256)     

In [ ]:
 # compile statement  when dense layers are removed
 # model.compile(
  #  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #true since no softmax used at output layer
  #  optimizer=keras.optimizers.Adam(learning_rate=3e-4),
   # metrics=["accuracy"], 
#)
#initializer = keras.initializers.he_normal()
model1.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #tensorboard = TensorBoard(os.path.join('energylog',f'{batchsize}_{time.time()}'))
  #@measure_energy(handler=csv_handler, domains=[NvidiaGPUDomain(0)])
  #@measure_energy(handler=csv_handler,domains=[RaplPackageDomain(1), NvidiaGPUDomain(0),RaplDramDomain(0)])
  #def foo():
model1.fit(ds_train,  epochs=25, validation_data=ds_test, callbacks = [tensorboard])
  #for _ in range(1):
   #   foo()
  #..
#model.fit(ds_train,  epochs=10, validation_data=ds_test)
test_loss, test_acc = model1.evaluate(ds_test, verbose=2)

In [ ]:
#initializer = keras.initializers.he_normal()
model1.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #tensorboard = TensorBoard(os.path.join('energylog',f'{batchsize}_{time.time()}'))
  #@measure_energy(handler=csv_handler, domains=[NvidiaGPUDomain(0)])
  #@measure_energy(handler=csv_handler,domains=[RaplPackageDomain(1), NvidiaGPUDomain(0),RaplDramDomain(0)])
  #def foo():
model1.fit(ds_train,  epochs=25, validation_data=ds_test, callbacks = [tensorboard])
  #for _ in range(1):
   #   foo()
  #..
#model.fit(ds_train,  epochs=10, validation_data=ds_test)
test_loss, test_acc = model1.evaluate(ds_test, verbose=2)

7. GROUP CONV + DILATED CONV

In [12]:
#group+dil      8. group =32 #wait okay?  MODEL 8
#alexNet
model1 = keras.models.Sequential([
  keras.layers.Conv2D(filters=64, kernel_size=(11,11), strides=(4,4), input_shape=(224,224,3)),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),


  #keras.layers.DepthwiseConv2D(depth_multiplier=8,kernel_size=5,strides=(1,1),padding='same'),
  keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1),groups= 32,  padding="same",dilation_rate=2),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  

  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1),groups= 32,  padding="same",dilation_rate=2),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1),groups= 32,  padding="same",dilation_rate=2),
  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1),groups= 32, padding="same",dilation_rate=2),
  #keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  
  #keras.layers.Conv2D(filters=nclass,kernel_size = (1,1), activation='relu', padding="same"), #add number of classes here
  #keras.layers.AvgPool2D(pool_size=(5,5))

  keras.layers.Flatten(),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(10, activation='softmax')
   
  ])
model1.summary()

 # compile statement  when dense layers are removed
 # model.compile(
  #  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #true since no softmax used at output layer
  #  optimizer=keras.optimizers.Adam(learning_rate=3e-4),
   # metrics=["accuracy"], 
#)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 54, 54, 64)        23296     
                                                                 
 batch_normalization_38 (Bat  (None, 54, 54, 64)       256       
 chNormalization)                                                
                                                                 
 activation_38 (Activation)  (None, 54, 54, 64)        0         
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 26, 26, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_32 (Conv2D)          (None, 26, 26, 256)       768       
                                                                 
 batch_normalization_39 (Bat  (None, 26, 26, 256)     

In [ ]:
#initializer = keras.initializers.he_normal()
model1.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #tensorboard = TensorBoard(os.path.join('energylog',f'{batchsize}_{time.time()}'))
  #@measure_energy(handler=csv_handler, domains=[NvidiaGPUDomain(0)])
  #@measure_energy(handler=csv_handler,domains=[RaplPackageDomain(1), NvidiaGPUDomain(0),RaplDramDomain(0)])
  #def foo():
model1.fit(ds_train,  epochs=25, validation_data=ds_test, callbacks = [tensorboard])
  #for _ in range(1):
   #   foo()
  #..
#model.fit(ds_train,  epochs=10, validation_data=ds_test)
test_loss, test_acc = model1.evaluate(ds_test, verbose=2)

8.DWC+DILATED+PWC+GROUP

In [13]:

 #lcrm v4  lcrmv4- depthwise dilation, pointwise group
nclasses=10
#nclasses =1000
def my_model1():
    #entry flow
    input = keras.Input(shape=(224,224,3))
    x = layers.Conv2D(64,kernel_size=11,strides=(4,4))(input)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
    
    
    x = layers.DepthwiseConv2D(depth_multiplier=4,kernel_size=5,strides=(1,1),dilation_rate=(2, 2),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x1 = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)

      #middle loop
    x = layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),dilation_rate=(2, 2),padding='same')(x1)
    x = layers.BatchNormalization()(x)
    x2= layers.Activation('relu')(x)

    x = layers.Conv2D(128,kernel_size=1,strides=(1,1),padding='same',groups=32)(x1)
    x = layers.BatchNormalization()(x)
    x3= layers.Activation('relu')(x)
    x = layers.Concatenate()([x2,x3])

    x = layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),dilation_rate=(2, 2),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    
    #Exit flow
    x = layers.Conv2D(256,kernel_size=1,strides=(1,1),padding='same', groups=32)(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)

    x=keras.layers.Flatten()(x)
    x=keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)

    x= keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(10, activation='softmax')(x)
    model = keras.Model(input, output)
    return model

model1=my_model1()
model1.summary()



Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_36 (Conv2D)             (None, 54, 54, 64)   23296       ['input_4[0][0]']                
                                                                                                  
 batch_normalization_43 (BatchN  (None, 54, 54, 64)  256         ['conv2d_36[0][0]']              
 ormalization)                                                                                    
                                                                                            

In [ ]:
model1.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #tensorboard = TensorBoard(os.path.join('energylog',f'{batchsize}_{time.time()}'))
  #@measure_energy(handler=csv_handler, domains=[NvidiaGPUDomain(0)])
  #@measure_energy(handler=csv_handler,domains=[RaplPackageDomain(1), NvidiaGPUDomain(0),RaplDramDomain(0)])
    #def foo():
model1.fit(ds_train,  epochs=25, validation_data=ds_test, callbacks = [tensorboard])
  #for _ in range(1):
   #   foo()
  #..
#model.fit(ds_train,  epochs=10, validation_data=ds_test)
test_loss, test_acc = model1.evaluate(ds_test, verbose=2)
  #def foo():


9. DWC+DILATED+PWC

In [14]:
##lcrm v3 lcrmv3- depthwise dilation, pointwise only hm, 
nclasses=10
#nclasses =1000
def my_model1():
    #entry flow
    input = keras.Input(shape=(224,224,3))
    x = layers.Conv2D(64,kernel_size=11,strides=(4,4))(input)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
    
    
    x = layers.DepthwiseConv2D(depth_multiplier=4,kernel_size=5,strides=(1,1),dilation_rate=(2, 2),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x1 = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)

      #middle loop
    x = layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),dilation_rate=(2, 2),padding='same')(x1)
    x = layers.BatchNormalization()(x)
    x2= layers.Activation('relu')(x)

    x = layers.Conv2D(128,kernel_size=1,strides=(1,1),padding='same')(x1)
    x = layers.BatchNormalization()(x)
    x3= layers.Activation('relu')(x)
    x = layers.Concatenate()([x2,x3])

    x = layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),dilation_rate=(2, 2),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    
    #Exit flow
    x = layers.Conv2D(256,kernel_size=1,strides=(1,1),padding='same')(x)
    x = layers.BatchNormalization()(x)
    x= layers.Activation('relu')(x)
    x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)

    x=keras.layers.Flatten()(x)
    x=keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)

    x= keras.layers.Dense(1024, activation='relu')(x)
    x= keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(10, activation='softmax')(x)
    model = keras.Model(input, output)
    return model

model1=my_model1()
model1.summary()



Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_39 (Conv2D)             (None, 54, 54, 64)   23296       ['input_5[0][0]']                
                                                                                                  
 batch_normalization_49 (BatchN  (None, 54, 54, 64)  256         ['conv2d_39[0][0]']              
 ormalization)                                                                                    
                                                                                            

In [ ]:
model1.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #tensorboard = TensorBoard(os.path.join('energylog',f'{batchsize}_{time.time()}'))
  #@measure_energy(handler=csv_handler, domains=[NvidiaGPUDomain(0)])
  #@measure_energy(handler=csv_handler,domains=[RaplPackageDomain(1), NvidiaGPUDomain(0),RaplDramDomain(0)])
    #def foo():
model1.fit(ds_train,  epochs=25, validation_data=ds_test, callbacks = [tensorboard])
  #for _ in range(1):
   #   foo()
  #..
#model.fit(ds_train,  epochs=10, validation_data=ds_test)
test_loss, test_acc = model1.evaluate(ds_test, verbose=2)
  #def foo():


10. TRANSPOSE CONVOLUTION ONLY

In [16]:
#transpose
#COMPARING THE ACTUAL ALEX NET 
#alexNet
model1 = keras.models.Sequential([
  keras.layers.Conv2D(filters=64, kernel_size=(11,11), strides=(4,4), input_shape=(224,224,3)),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),


  #keras.layers.DepthwiseConv2D(depth_multiplier=8,kernel_size=5,strides=(1,1),padding='same'),
  keras.layers.Conv2DTranspose(filters=256, kernel_size=(5,5), strides=(1,1), padding="same"),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  

  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.Conv2DTranspose(filters=384, kernel_size=(3,3), strides=(1,1), padding="same"),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2DTranspose(filters=384, kernel_size=(3,3), strides=(1,1), padding="same"),
  #keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),

  keras.layers.Conv2DTranspose(filters=256, kernel_size=(3,3), strides=(1,1), padding="same"),
  #keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Activation('relu'),
  keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
  
  #keras.layers.Conv2D(filters=nclass,kernel_size = (1,1), activation='relu', padding="same"), #add number of classes here
  #keras.layers.AvgPool2D(pool_size=(5,5))

  keras.layers.Flatten(),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.5),

  keras.layers.Dense(10, activation='softmax')
   
  ])
model1.summary()

 # compile statement  when dense layers are removed
 # model.compile(
  #  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #true since no softmax used at output layer
  #  optimizer=keras.optimizers.Adam(learning_rate=3e-4),
   # metrics=["accuracy"], 
#)
#initializer = keras.initializers.he_normal()
model1.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #tensorboard = TensorBoard(os.path.join('energylog',f'{batchsize}_{time.time()}'))
  #@measure_energy(handler=csv_handler, domains=[NvidiaGPUDomain(0)])
  #@measure_energy(handler=csv_handler,domains=[RaplPackageDomain(1), NvidiaGPUDomain(0),RaplDramDomain(0)])

    #def foo():
model1.fit(ds_train,  epochs=25, validation_data=ds_test, callbacks = [tensorboard])
  #for _ in range(1):
   #   foo()
  #..
#model.fit(ds_train,  epochs=10, validation_data=ds_test)
test_loss, test_acc = model1.evaluate(ds_test, verbose=2)





Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_43 (Conv2D)          (None, 54, 54, 64)        23296     
                                                                 
 batch_normalization_60 (Bat  (None, 54, 54, 64)       256       
 chNormalization)                                                
                                                                 
 activation_60 (Activation)  (None, 54, 54, 64)        0         
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 26, 26, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 26, 26, 256)      409856    
 ranspose)                                                       
                                                      